In [ ]:
! pip install datasets

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install transformers[torch] -U

In [ ]:
import json
with open('C:/Users/surya/Downloads/chit_chat_generator/chit_chat.json', 'r') as file:
    data = json.load(file)

In [ ]:
import pandas as pd
import csv
import re
import string

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s\.,!?]', '', text)
    PUNCT_TO_REMOVE = string.punctuation
    clean_text = text.translate(str.maketrans("", "", PUNCT_TO_REMOVE))
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

In [ ]:
formatted_data = []

for key, value in data.items():
    input_text = ""
    target_text = ""
    first_sender = None
    next_sender = None

    for messages in value["messages"]:
        for j, msg in enumerate(messages):
          msg_text = clean_text(msg["text"]).strip()
          if first_sender is None:
            first_sender = msg["sender"]

          if msg["sender"] == first_sender:
            if input_text == "":
              input_text = msg_text
            if j < len(messages) - 1:
              next_sender = messages[j + 1]["sender"]
            else:
              next_sender = None

            if msg["sender"] == next_sender:
              input_text = input_text + " </s> " + clean_text(messages[j + 1]["text"]).strip()
              continue
          elif msg["sender"] != first_sender:
            if target_text == "":
              target_text = msg_text
            if j < len(messages) - 1:
              next_sender = messages[j + 1]["sender"]
            else:
              next_sender = None

            if msg["sender"] == next_sender:
              target_text = target_text + " </s> " + clean_text(messages[j + 1]["text"]).strip()
              continue

            formatted_data.append({'input': input_text.strip() + " </s> ", 'target': target_text.strip() + " </s> "})
            input_text = ""
            target_text = ""

In [ ]:
formatted_data_out = pd.DataFrame(formatted_data)
formatted_data_out.to_csv('C:/Users/surya/Downloads/chit_chat_generator/train_data.txt', sep='\t', index=False, header=False)
print("Formatted data stored in train_data.txt")

Formatted data stored in train_data.txt


In [ ]:
formatted_dataset = {
    "input_text": [pair["input"] for pair in formatted_data],
    "output_text": [pair["target"] for pair in formatted_data]
}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = "kssumanth6/t5_small_chit_chat_generator_v3"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer = T5Tokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
def tokenize_function(talks):
    tokenized_inputs = tokenizer(talks['input_text'], padding='max_length', truncation=True, max_length=256)
    tokenized_outputs = tokenizer(talks['output_text'], padding='max_length', truncation=True, max_length=256)
    return {'input_ids': tokenized_inputs['input_ids'], 'attention_mask': tokenized_inputs['attention_mask'], 'labels': tokenized_outputs['input_ids']}

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict(formatted_dataset)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_eval_split = tokenized_datasets.train_test_split(test_size=0.2, shuffle=False)
train_dataset = train_eval_split['train']
eval_dataset = train_eval_split['test']

val_test_split = eval_dataset.train_test_split(test_size = 0.5, shuffle=False)
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

Map:   0%|          | 0/66853 [00:00<?, ? examples/s]

C:\Users\surya\anaconda3\Lib\site-packages\transformers\models\t5\tokenization_t5.py:290: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [ ]:
!pip install datasets sentencepiece transformers rouge_score bert_score huggingface_hub
!pip install scikit-learn scipy matplotlib --upgrade


In [ ]:
!pip install sacrebleu
!pip install accelerate -U

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from sacrebleu import corpus_bleu
import torch
import torch.nn as nn
from transformers import pipeline

In [ ]:
!pip3 install rouge_score nltk absl-py transformers evaluate sentencepiece
!pip3 install bleurt

ERROR: Could not find a version that satisfies the requirement bleurt (from versions: none)
ERROR: No matching distribution found for bleurt


In [ ]:
test_dataset = pd.DataFrame(test_dataset)

In [ ]:
test_dataset[:10]

,input_text,output_text,input_ids,attention_mask,labels
0,i am graduating so i am looking for a job i wa...,that sounds super exciting not the not getting...,"[3, 23, 183, 20220, 53, 78, 3, 23, 183, 479, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[24, 2993, 1355, 2949, 59, 8, 59, 652, 3, 9, 6..."
1,are you and your husband nearing graduation </s>,i have a year left but hes double majoring and...,"[33, 25, 11, 39, 2553, 1084, 53, 14027, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[3, 23, 43, 3, 9, 215, 646, 68, 3, 88, 7, 1486..."
2,oh wow </s> double majoring sounds like a lot ...,i personally believe he is ray </s> oops </s> ...,"[3, 32, 107, 2275, 210, 1, 1486, 779, 53, 2993...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 23, 4828, 857, 3, 88, 19, 3, 2866, 1, 3, 6..."
3,hi there </s>,hey whats up </s>,"[7102, 132, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 13133, 125, 7, 95, 1, 0, 0, 0, 0, 0, 0, 0,..."
4,not much just packing for a trip what about yo...,im just layin around being lazy because i dona...,"[59, 231, 131, 12935, 21, 3, 9, 1469, 125, 81,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[256, 131, 8260, 77, 300, 271, 19743, 250, 3, ..."
5,oh thats awesome thats kind of funny because i...,ugh they call me every day i had to block the ...,"[3, 32, 107, 24, 7, 2879, 24, 7, 773, 13, 6613...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 14439, 79, 580, 140, 334, 239, 3, 23, 141,..."
6,oh really i live in a small town so we dont re...,cool do you play violin </s> and its nice to m...,"[3, 32, 107, 310, 3, 23, 619, 16, 3, 9, 422, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1633, 103, 25, 577, 24325, 1, 11, 165, 1245, ..."
7,yeah i actually do play violin </s> how did yo...,just lucky i guess i was originally going to g...,"[17945, 3, 23, 700, 103, 577, 24325, 1, 149, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[131, 5722, 3, 23, 3382, 3, 23, 47, 5330, 352,..."
8,i have been playing for 12 years </s> do you p...,wow thats awesome </s> i used to play violin a...,"[3, 23, 43, 118, 1556, 21, 586, 203, 1, 103, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2275, 210, 24, 7, 2879, 1, 3, 23, 261, 12, 57..."
9,thanks and wow 8 years on piano is super good ...,ill be a freshman what about you </s>,"[2049, 11, 2275, 210, 505, 203, 30, 8355, 19, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 1092, 36, 3, 9, 22857, 125, 81, 25, 1, 0, ..."


In [ ]:
for index, row in test_dataset[0:10].iterrows():
   input_text, label = row['input_text'], row['output_text']
   print(input_text)
   print(label)

i am graduating so i am looking for a job i was going to be a river rafting guide for fun before getting a full time job but they didnt hire me so i think ill just </s> mountain bike a bunch </s> 
that sounds super exciting not the not getting a job part yay for graduation </s> 
are you and your husband nearing graduation </s> 
i have a year left but hes double majoring and has three more to go </s> 
oh wow </s> double majoring sounds like a lot of work i have a minor but mostly because i switched majors but had only one class left for a minor </s> 
i personally believe he is ray </s> oops </s> i personally believe he is crazy but he enjoys it </s> 
hi there </s> 
hey whats up </s> 
not much just packing for a trip what about you </s> 
im just layin around being lazy because i donated blood today haha </s> where are you going on your trip </s> 
oh thats awesome thats kind of funny because i just got called by red cross to donate blood d </s> and i leave tomorrow at 1030 </s> can i ask 

In [ ]:
def generate_text(prompt_text, model, tokenizer, device):

    input_ids = tokenizer(prompt_text, return_tensors='pt', padding="max_length", max_length=256, truncation=True)

    input_ids = input_ids.to(device)

    outputs = model.generate(input_ids=input_ids['input_ids'], max_length=50, do_sample=True, num_beams=5, no_repeat_ngram_size=2)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

In [ ]:
len(test_dataset)

6686

In [ ]:
original_sentences = []
generated_sentences = []
start = 0
end = 50
for index, row in test_dataset[start:end].iterrows():
    input_text, label = row['input_text'], row['output_text']
    generated_sentences.append(generate_text(input_text, model, tokenizer, device))
    original_sentences.append(label)

C:\Users\surya\anaconda3\Lib\site-packages\transformers\models\t5\tokenization_t5.py:290: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [ ]:
len(original_sentences), len(generated_sentences)

(50, 50)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

original_tokenized = [sentence.split() for sentence in original_sentences]
generated_tokenized = [sentence.split() for sentence in generated_sentences]

bleu1_score = corpus_bleu([[ref] for ref in original_tokenized], generated_tokenized, weights=(1, 0, 0, 0))

bleu1_score


0.139221177851228

In [ ]:
from bert_score import score

P, R, F1 = score(cands=generated_sentences, refs=original_sentences, lang="en", verbose=True)

F1.mean().item()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.55 seconds, 91.21 sentences/sec


0.8250133395195007

In [ ]:
from evaluate import load

rouge = load('rouge')

rouge_score = rouge.compute(predictions=generated_sentences, references=original_sentences)

rouge_score

{'rouge1': 0.13979108972702375,
 'rouge2': 0.015832660442918162,
 'rougeL': 0.11590930668026106,
 'rougeLsum': 0.11627967010708015}

In [ ]:
input_ids_original = tokenizer(original_sentences, padding="max_length", max_length=256, return_tensors="pt", truncation=True)

input_ids_generated = tokenizer(generated_sentences, padding="max_length", max_length=256, return_tensors="pt", truncation=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_ids_original = input_ids_original.to(device)
input_ids_generated = input_ids_generated.to(device)

outputs = model(input_ids=input_ids_original["input_ids"], labels=input_ids_generated["input_ids"])

loss = outputs.loss.item()
perplexity = torch.exp(outputs.loss).item()

print(f"Loss: {loss}")
print(f"Perplexity: {perplexity}")

C:\Users\surya\anaconda3\Lib\site-packages\transformers\models\t5\tokenization_t5.py:290: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Loss: 0.15610608458518982
Perplexity: 1.1689502000808716
